<a href="https://colab.research.google.com/github/pravinjaju1005/pravinjaju1005/blob/main/nse_index_closing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
# !pip install nselib
# !pip install pandas_market_calendars

In [72]:
# import nselib
# from nselib import capital_market

In [73]:
# capital_market.market_watch_all_indices

In [74]:
# from nsepython import *

# # Get all indices (includes NIFTY, BANKNIFTY, etc.)
# indices = nse_index()
# indices.head()

In [75]:
# # prompt: Using dataframe indices: get all indices symbol

# # Display all index names
# indices_list = indices['indexName'].unique()
# indices_list[:5]

In [76]:
import pandas as pd
import datetime as dt
import json
import zipfile
from io import BytesIO, StringIO
import logging
import xml.etree.ElementTree as ET
from nselib.libutil import *
from nselib.constants import *

In [77]:
def equity_list():
    """
    get list of all equity available to trade in NSE
    :return: pandas data frame
    """
    origin_url = "https://nsewebsite-staging.nseindia.com"
    url = "https://archives.nseindia.com/content/equities/EQUITY_L.csv"
    file_chk = nse_urlfetch(url,origin_url=origin_url)
    if file_chk.status_code != 200:
        raise FileNotFoundError(f" No data equity list available")
    try:
        data_df = pd.read_csv(BytesIO(file_chk.content))
    except Exception as e:
        raise FileNotFoundError(f' Equity List not found :: NSE error : {e}')
    data_df = data_df[['SYMBOL', 'NAME OF COMPANY', ' SERIES', ' DATE OF LISTING', ' FACE VALUE']]
    return data_df

In [78]:
# equity_list()

In [79]:
def market_watch_all_indices():
    """
    Market Watch - Indices of the day in data frame
    :return: pd.DataFrame
    """
    origin_url = "https://nsewebsite-staging.nseindia.com"
    url = "https://www.nseindia.com/api/allIndices"
    data_json = nse_urlfetch(url, origin_url=origin_url).json()
    data_df = pd.DataFrame(data_json['data'])
    return data_df[['key', 'index', 'indexSymbol', 'last', 'variation', 'percentChange', 'open', 'high', 'low',
                   'previousClose', 'yearHigh', 'yearLow', 'pe', 'pb', 'dy', 'declines', 'advances', 'unchanged',
                   'perChange365d', 'perChange30d', 'previousDay', 'oneWeekAgo', 'oneMonthAgo', 'oneYearAgo']]


In [80]:
# market_watch_all_indices()

In [81]:
def fetch_daily_index(date: str):
    """
    date in format: 'DDMMYYYY'
    """
    url = f"https://archives.nseindia.com/content/indices/ind_close_all_{date}.csv"
    try:
        df = pd.read_csv(url)
        return df
    except:
        return None

# Example: fetch 1st week of Jan 2023
from datetime import datetime, timedelta

start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 1, 7)

all_dfs = []
while start_date <= end_date:
    date_str = start_date.strftime("%d%m%Y")
    df = fetch_daily_index(date_str)
    if df is not None:
        df['Date'] = start_date.strftime("%Y-%m-%d")
        all_dfs.append(df)
    start_date += timedelta(days=1)

# Merge into one DataFrame
final_df = pd.concat(all_dfs, ignore_index=True)
# print(final_df.head())


In [82]:
pivot_df = final_df.pivot(index='Index Date', columns='Index Name', values='Closing Index Value')

In [83]:
pivot_df = pivot_df.sort_index()
pivot_df.head()

Index Name,India VIX,NIFTY Alpha Low-Volatility 30,NIFTY Alpha Quality Low-Volatility 30,NIFTY Alpha Quality Value Low-Volatility 30,NIFTY LargeMidcap 250,NIFTY Midcap 100,NIFTY Midcap150 Quality 50,NIFTY Quality Low-Volatility 30,NIFTY SME EMERGE,NIFTY Smallcap 100,...,Nifty500 Equal Weight,Nifty500 LargeMidSmall Equal-Cap Weighted,Nifty500 Low Volatility 50,Nifty500 Momentum 50,Nifty500 Multicap 50:25:25,Nifty500 Multicap India Manufacturing 50:30:20,Nifty500 Multicap Infrastructure 50:30:20,Nifty500 Multicap Momentum Quality 50,Nifty500 Quality 50,Nifty500 Shariah
Index Date,,,,,,,,,,,,,,,,,,,,,
01-01-2025,14.51,27346.10,22759.30,20388.25,15786.10,57450.90,24490.65,16933.45,16821.94,18959.80,...,14318.20,17697.35,20115.13,57480.15,15880.50,14805.60,13758.35,45301.60,19761.50,7489.70
02-01-2025,13.74,27761.30,23116.30,20712.50,16009.05,58108.20,24644.55,17267.85,16908.26,19080.35,...,14446.20,17896.30,20628.16,58382.05,16084.15,15018.85,13892.05,45894.70,19925.14,7570.85
03-01-2025,13.54,27513.40,23002.05,20545.45,15941.35,57931.05,24568.90,17223.05,17125.48,19033.70,...,14439.05,17834.30,20554.66,58041.35,16018.45,14963.95,13847.35,45568.15,19871.92,7553.80
06-01-2025,15.65,27158.25,22573.95,20234.25,15588.20,56366.95,24089.50,16977.85,16694.53,18425.25,...,14048.55,17399.65,20282.28,56481.05,15649.10,14639.60,13474.00,44577.20,19340.41,7428.00
07-01-2025,14.66,27146.10,22637.90,20256.80,15679.25,56869.30,24262.80,16975.30,16911.98,18673.45,...,14197.10,17549.35,20321.81,56557.75,15761.85,14760.65,13612.95,44757.65,19501.81,7456.60
